# BVS CMDB Template Files Maker
`VERSION: 1.3`: se incluye el feature `U_Sla` en builder para incluirlo en la Linea de contrato y de esta forma hacer una diferenciación por tipo de contrato.
<br>De esta forma esta nueva versión permite diferenciar un SLA por cada linea de contrato. 
<br>Esta funcionalidad aplica de la siguiente forma: 
   * Para el caso de VBR: Toma SLA directamente desde el quote.
   * Para el caso de BCARE: Toma SLA definido en builder.xls

## Indice
> * CARGA DE LIBRERIAS
> * FUNCIONES
> * PRE BUILDER CABECERA_CONTRATO
> * PRE BUILDER TARJETA_EQUIPO
> * PRE BUILDER LINEAS_CONTRATO

## Entradas
> * PRE BUILDER TARJETA_EQUIPO
> * PRE BUILDER LINEAS_CONTRATO

## Salidas
> * `CABECERA_CONTRATO.txt`
> * `TARJETA_EQUIPO.csv`
> * `LINEAS_CONTRATO.csv`

## CARGA DE LIBRERIAS

In [1]:
import pandas as pd
import numpy as np

# Para visualizar todos los features del data frame
pd.set_option('display.max_columns', None)

## FUNCIONES

In [2]:
def add_columns_to_df(df, columna, dato):
    '''Agrega columnas y su correspondiente dato de forma masiva dentro de un data frame'''
    df[columna] = dato

def add_index_to_df(df, columna):
    '''Agrega una columna adicional de indice comenzando desde 1'''
    df[columna] = df.index+1

def column_filter(df, filtro):
    '''Aplica filtro de columnas en un data frame'''
    return pd.DataFrame(df[filtro])

In [3]:
def column_names_update(df, col_definitivas):
    '''Actualizacion de columnas con nombres definitivos. Requiere que el data frame se encuentre previamente ordenado y 
    con la cantidad de columnas correctas'''
    df.columns = col_definitivas
    return df

In [4]:
def nan_cleaner(df):
    '''Eliminación de NaN's'''
    #df.replace(np.nan, "", inplace=True)
    df.replace({pd.NaT: "", np.NaN: ""}, inplace=True)
    #df.replace({pd.NaT: pd.to_datetime("", format='%Y%m%d', errors="ignore"),  np.NaN: ""}, inplace=True)
    return df

In [5]:
def add_cabecera(df, col_cabecera):
    df.columns = pd.MultiIndex.from_tuples(zip(col_cabecera, df.columns))
    return df

In [6]:
def add_column_format(df, col_filtro, col_cabecera, col_definitivas):
    '''Funición para limpiar NaN's, renombrar columnas con la descripción final y agregado de cabecera adicional'''
    df = column_filter(df, col_filtro) # Filtrado de columnas
    df = column_names_update(df, col_definitivas) # Actualizacion de columnas con nombres definitivos
    df = nan_cleaner(df)
    df.columns = pd.MultiIndex.from_tuples(zip(col_cabecera, df.columns)) # Agregado de cabecera
    print("MSG: Data-Frame listo para procesar en formato CSV")
    return df

# 1. SCRIPT CODE
 - ccr_df (Cabecera Contrato Raw) --> bvs.cmdb.template_builder
 - ter_df (Tarjeta de Equipo Raw) --> quote, bvs.cmdb.template_builder
 - lcr_df (Lineas de Contrato Raw) --> ter_df, bvs.cmdb.template_builder

### 1.1 PRE BUILDER CABECERA_CONTRATO
   * ccr_df: cabecera contrato raw data-frame

In [7]:
# Lectura del archivo CABECERA_CONTRATO previamente completado por el CMDB ADMIN.
#ccr: cabecera_contrato_raw_df
#ccr_df = pd.DataFrame(pd.read_excel('/cmdb.income.files/CABECERA_CONTRATO.xlsx', skiprows=1 ,encoding='ISO-8859-1')) #for macOS
ccr_df = pd.DataFrame(pd.read_excel('./cmdb.income.files/bvs.cmdb.template_builder.xlsx', skiprows=1))
# ccr_df = pd.DataFrame(pd.read_excel('CABECERA_CONTRATO_RAW.xlsx', skiprows=1 ,encoding='ISO-8859-1'))

In [8]:
# Generación de nombres de columnas
 # cc_cabecera: es un requerimiento del SAP, se traduce como un DF con dos headers
cc_cabecera = ["ContractID", "CustomerCode", "Status", "ContractType", "Renewal", "StartDate", "EndDate", "Description",
            "ServiceType", "U_ServType", "U_Provider", "U_ServiceName"]
cc_filtro_columnas = ["ContractID", "CstmrCode", "Status", "CntrcType", "Renewal", "StartDate", "EndDate", "Descriptio", "SrvcType", "U_ServType", "U_Provider", "U_ServiceName"]
cc_columnas_definitivas = ["ContractID", "CstmrCode", "Status", "CntrcType", "Renewal", "StartDate", "EndDate", "Descriptio", "SrvcType", "U_ServType", "U_Provider", "U_ServiceName"]


In [9]:
ccr_df

,ContractID,CstmrCode,Status,CntrcType,Renewal,StartDate,EndDate,Descriptio,SrvcType,U_ServType,U_Provider,U_ServiceName,U_OV,U_statusVendor,U_Sla
0,0,CEXT00000005,scs_Approved,ct_SerialNumber,tNO,2021-02-01,2022-01-31,NaN,bst_Regular,NaN,CISCO,SMARTNET,OV 2236 BVS GROUP CORP.,ACTIVE,


#### 1.1.1 FORMATEO Y LIMPEZA DE DATOS

In [10]:
ccr_total_df = ccr_df.copy()

In [11]:
ccr_total_df = add_column_format(ccr_total_df, cc_filtro_columnas, cc_cabecera, cc_columnas_definitivas)

MSG: Data-Frame listo para procesar en formato CSV


In [12]:
ccr_total_df

,ContractID,CustomerCode,Status,ContractType,Renewal,StartDate,EndDate,Description,ServiceType,U_ServType,U_Provider,U_ServiceName
,ContractID,CstmrCode,Status,CntrcType,Renewal,StartDate,EndDate,Descriptio,SrvcType,U_ServType,U_Provider,U_ServiceName
0,0,CEXT00000005,scs_Approved,ct_SerialNumber,tNO,2021-02-01,2022-01-31,,bst_Regular,,CISCO,SMARTNET


#### 1.1.2 CREACION DE CSV

In [13]:
# Salida en formato csv
ccr_total_df.to_csv("./cmdb.templates.ok/CABECERA_CONTRATO.csv", index = False)
# ccr_total_df.to_excel("./cmdb.templates.ok/Test_CABECERA_CONTRATO.xls", index = False)

### 1.2 PRE BUILDER TARJETA_EQUIPO
 - ter_df: (Tarjeta Equipo Raw) --> quote, bvs.cmdb.template_builder

NOTA: Un DF de esta tarjeta se utiliza para replicarlo por `ContractID` definido en CABECERA_CONTRATO

#### 1.2.1 LECTURA DE ARCHIVO FUENTE

In [14]:
# Lectura del quote Cisco (unicamente). Se descartan las 27 primeras líneas del header ("skiprows").
# ter_df = pd.DataFrame(pd.read_excel('quote.xlsx', skiprows=26,encoding='ISO-8859-1')) for macOS
#ter_df = pd.DataFrame(pd.read_excel('./cmdb.income.files/quote.xlsx', converters= {'Last Date of Support': pd.to_datetime}, skiprows=26))
#quote_df = pd.DataFrame(pd.read_excel('./cmdb.income.files/quote.xlsx', converters= {'Last Date of Support': pd.to_datetime}, skiprows=26))
quote_df = pd.DataFrame(pd.read_excel('./cmdb.income.files/quote.xlsx', skiprows=26))
#df1 = pd.read_excel(file, converters= {'Last Date of Support': pd.to_datetime})

In [15]:
# Copia del quote así lo reutilizamos para construit la tarjeta equipo
ter_df = quote_df.copy()

#### 1.2.2 CREACIÓN DE FILTROS Y COLUMNAS

In [16]:
#TODO: TRATAR DE INCLUIR EOS EN SAP *** HABLARLO CON LUCAS ***

# Generación de nombres de columnas
# filtro_columnas: son las columnas que nos interesan del DF fuente y se utilizarán para filtrar el mismo DF
# columnas_definitivas: son los nombres con los que deben llamarse en df_tarjeta_equipo
# cabecera: es un requerimiento del SAP, se traduce como un DF con dos headers

te_filtro_columnas = ["insID", "customer", "Instance Number", "PAK/Serial Number", "Product Number", "Product Description", "End Customer Address Line 1","End Customer Zip/Postal Code" ,"End Customer City", "End Customer Country", "status"]
te_columnas_definitivas = ["insID","customer", "manufSN", "internalSN", "itemCode", "itemName", "street", "zip", "city", "county", "status"]
te_cabecera = ["EquipmentCardNum", "CustomerCode", "ManufacturerSerialNum", "InternalSerialNum", "ItemCode", "ItemDescription","Street", "ZipCode", "City", "County", "StatusOfSerialNumber"]

#### 1.2.3 AGREGADO DE COLUMNAS

In [17]:
# Agregado de features faltantes y precargado de valores iniciales. Este data-gframe se utilizará como template para completar tarjeta de equipo

# Lectura de variables provistas en ccr_df que luego se utilizan para precargar el template ter_df
customer = ccr_df.loc[0,"CstmrCode"]
status = ccr_df.loc[0,"Status"]

# Precarga de valores leidos previamente
add_index_to_df(ter_df,"insID")
add_columns_to_df(ter_df,"customer", customer)
add_columns_to_df(ter_df,"status", status)

In [18]:
ter_df.head(2)

,Product Number,Product Description,Last Date of Support,PAK/Serial Number,Instance Number,Major/Minor,Parent Instance Number,Line Type,Host ID/Mac ID,Service Level,Service Level Description,SKU,Service Type,Takeover Line,Source Contract Number,Target Contract Number,Start Date,End Date,Quantity,Quantity UOM,Revenue Source Code,Price UOM,Unit List Price,Prorated List Price,Total Discounts,Extended Net Price,Credit Adjustment,End Customer GU ID,End Customer GU Name,End Customer ID,End Customer Name,End Customer Address Line 1,End Customer Address Line 2,End Customer City,End Customer State,End Customer Country,End Customer Zip/Postal Code,Ship to Id,Ship to Name,Ship to Address Line 1,Ship to Address Line 2,Ship to City,Ship to State,Ship to Country,Ship to Zip/Postal Code,Ship to Contact First name,Ship to Contact Last name,Ship to Email,Ship to Phone Number,eDelivery Email Address,Tax Status,PO Number,SO Number,Service Price List,Product Category,Product List Price,Service List Factor,Product Family,Smart Account / Virtual Account,Initial Term (Months),Billing Frequency,Automatically Renews For,Subscription ID,Reference Serial Number,Reference Instance Number,insID,customer,status
0,UCSC-C220-M4S,"UCS C220 M4 SFF w/o CPU, mem, HD, PCIe, PSU, r...",2024-02-29,FCH2216V13P,5243285545,Major,5243285545,RENEW,NaN,SNT,SNTC 8X5XNBD,CON-SNT-C220M4S,TS,No,203061476,203061476,2021-02-01,2022-01-31,1,NaN,R,Annual,576.13,576.13,32,391.77,NaN,2353270,AMC NETWORKS INC,1011595200,AMC NETWORKS INC,CAMPILLO 2541,PARQUE CHAS,BUENOS AIRES,CIUDAD AUTONOMA DE BUENOS AIRES,AR,1427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-EXEMPT,OV 351 BVS TV SA,106467329,Global PriceList-Latin America,NaN,0,NaN,UCSC,AMC NETWORKS INC-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,1,CEXT00000005,scs_Approved
1,UCSC-RAILB-M4,Ball Bearing Rail Kit for C220 & C240 M4 & M5 ...,NaT,NaN,5243285574,Minor,5243285545,RENEW,NaN,SNT,SNTC 8X5XNBD,NaN,TS,No,203061476,203061476,2021-02-01,2022-01-31,1,NaN,R,NaN,0.00,0.00,0,0.00,NaN,2353270,AMC NETWORKS INC,1011595200,AMC NETWORKS INC,CAMPILLO 2541,PARQUE CHAS,BUENOS AIRES,CIUDAD AUTONOMA DE BUENOS AIRES,AR,1427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-EXEMPT,OV 351 BVS TV SA,106467329,Global PriceList-Latin America,NaN,0,NaN,UCSC,AMC NETWORKS INC-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,2,CEXT00000005,scs_Approved


#### 1.2.4 FORMATEO Y LIMPEZA DE DATOS

In [19]:
ter_df = add_column_format(ter_df, te_filtro_columnas, te_cabecera, te_columnas_definitivas)

MSG: Data-Frame listo para procesar en formato CSV


In [20]:
ter_df.head(2)

,EquipmentCardNum,CustomerCode,ManufacturerSerialNum,InternalSerialNum,ItemCode,ItemDescription,Street,ZipCode,City,County,StatusOfSerialNumber
,insID,customer,manufSN,internalSN,itemCode,itemName,street,zip,city,county,status
0,1,CEXT00000005,5243285545,FCH2216V13P,UCSC-C220-M4S,"UCS C220 M4 SFF w/o CPU, mem, HD, PCIe, PSU, r...",CAMPILLO 2541,1427,BUENOS AIRES,AR,scs_Approved
1,2,CEXT00000005,5243285574,,UCSC-RAILB-M4,Ball Bearing Rail Kit for C220 & C240 M4 & M5 ...,CAMPILLO 2541,1427,BUENOS AIRES,AR,scs_Approved


#### 1.2.5 CREACION DE CSV

In [21]:
# Salida en formato csv
ter_df.to_csv("./cmdb.templates.ok/TARJETA_EQUIPO.csv", index = False)
#ter_df.to_excel("./cmdb.templates.ok/Test_TARJETA_EQUIPO.xls", index = False)

### 1.3 PRE BUILDER LINEAS_CONTRATO
 - lcr_df (Lineas de Contrato Raw) --> ter_df, bvs.cmdb.template_builder

#### 1.3.1 LECTURA DE ARCHIVO FUENTE

In [22]:
# lcr_df es un df que se utilizará como "base" para la confección del total de las lineas de contrato
lcr_df = quote_df.copy()

#### 1.3.2 CREACIÓN DE FILTROS Y COLUMNAS

In [23]:
# Generación de nombres de columnas
# filtro_columnas: son las columnas requeridas del DF fuente
# columnas_definitivas: son los nombres con los que deben llamarse en df_tarjeta_equipo


lc_cabecera = ["ParentKey", "LineNum", "ItemCode", "ItemName", "ManufacturerSerialNum", "InternalSerialNum", "StartDate", "EndDate", "U_StatusVendor", "U_Sla", "U_OV", "U_EndCustomer", "U_VendorContract", "U_ServSKU", "U_EoS"]
#lc_filtro_columnas = ["ParentKey", "LineNum", "Product Number", "Product Description", "Instance Number", "PAK/Serial Number", "Start Date","End Date" ,"U_StatusVendor", "Service Level Description", "U_OV", "End Customer Name", "U_VendorContract", "SKU","Last Date of Support"]
lc_filtro_columnas = ["ParentKey", "LineNum", "Product Number", "Product Description", "Instance Number", "PAK/Serial Number", "Start Date","End Date" ,"U_StatusVendor", "Service Level Description", "U_OV", "End Customer Name", "Target Contract Number", "SKU","Last Date of Support"]
lc_columnas_definitivas = ["ParentKey", "LineNum", "itemCode", "itemName", "ManufSN", "InternalSN", "StartDate", "EndDate", "U_StatusVendor", "U_Sla", "U_OV", "U_EndCustomer", "U_VendorContract", "U_ServSKU", "U_EoS"]

In [24]:
lcr_df.head(2)


,Product Number,Product Description,Last Date of Support,PAK/Serial Number,Instance Number,Major/Minor,Parent Instance Number,Line Type,Host ID/Mac ID,Service Level,Service Level Description,SKU,Service Type,Takeover Line,Source Contract Number,Target Contract Number,Start Date,End Date,Quantity,Quantity UOM,Revenue Source Code,Price UOM,Unit List Price,Prorated List Price,Total Discounts,Extended Net Price,Credit Adjustment,End Customer GU ID,End Customer GU Name,End Customer ID,End Customer Name,End Customer Address Line 1,End Customer Address Line 2,End Customer City,End Customer State,End Customer Country,End Customer Zip/Postal Code,Ship to Id,Ship to Name,Ship to Address Line 1,Ship to Address Line 2,Ship to City,Ship to State,Ship to Country,Ship to Zip/Postal Code,Ship to Contact First name,Ship to Contact Last name,Ship to Email,Ship to Phone Number,eDelivery Email Address,Tax Status,PO Number,SO Number,Service Price List,Product Category,Product List Price,Service List Factor,Product Family,Smart Account / Virtual Account,Initial Term (Months),Billing Frequency,Automatically Renews For,Subscription ID,Reference Serial Number,Reference Instance Number
0,UCSC-C220-M4S,"UCS C220 M4 SFF w/o CPU, mem, HD, PCIe, PSU, r...",2024-02-29,FCH2216V13P,5243285545,Major,5243285545,RENEW,NaN,SNT,SNTC 8X5XNBD,CON-SNT-C220M4S,TS,No,203061476,203061476,2021-02-01,2022-01-31,1,NaN,R,Annual,576.13,576.13,32,391.77,NaN,2353270,AMC NETWORKS INC,1011595200,AMC NETWORKS INC,CAMPILLO 2541,PARQUE CHAS,BUENOS AIRES,CIUDAD AUTONOMA DE BUENOS AIRES,AR,1427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-EXEMPT,OV 351 BVS TV SA,106467329,Global PriceList-Latin America,NaN,0,NaN,UCSC,AMC NETWORKS INC-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN
1,UCSC-RAILB-M4,Ball Bearing Rail Kit for C220 & C240 M4 & M5 ...,NaT,NaN,5243285574,Minor,5243285545,RENEW,NaN,SNT,SNTC 8X5XNBD,NaN,TS,No,203061476,203061476,2021-02-01,2022-01-31,1,NaN,R,NaN,0.00,0.00,0,0.00,NaN,2353270,AMC NETWORKS INC,1011595200,AMC NETWORKS INC,CAMPILLO 2541,PARQUE CHAS,BUENOS AIRES,CIUDAD AUTONOMA DE BUENOS AIRES,AR,1427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-EXEMPT,OV 351 BVS TV SA,106467329,Global PriceList-Latin America,NaN,0,NaN,UCSC,AMC NETWORKS INC-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN


#### 1.3.3 AGREGADO DE COLUMNAS

In [25]:
# Agregado de features faltantes y precargado de valores iniciales NaN´s.

add_index_to_df(lcr_df,"LineNum")
add_columns_to_df(lcr_df,"ParentKey", "")
#add_columns_to_df(lcr_df,"U_VendorContract", "")
add_columns_to_df(lcr_df,"U_StatusVendor", "")
add_columns_to_df(lcr_df,"U_OV", "")

In [26]:
lcr_df.head(2)

,Product Number,Product Description,Last Date of Support,PAK/Serial Number,Instance Number,Major/Minor,Parent Instance Number,Line Type,Host ID/Mac ID,Service Level,Service Level Description,SKU,Service Type,Takeover Line,Source Contract Number,Target Contract Number,Start Date,End Date,Quantity,Quantity UOM,Revenue Source Code,Price UOM,Unit List Price,Prorated List Price,Total Discounts,Extended Net Price,Credit Adjustment,End Customer GU ID,End Customer GU Name,End Customer ID,End Customer Name,End Customer Address Line 1,End Customer Address Line 2,End Customer City,End Customer State,End Customer Country,End Customer Zip/Postal Code,Ship to Id,Ship to Name,Ship to Address Line 1,Ship to Address Line 2,Ship to City,Ship to State,Ship to Country,Ship to Zip/Postal Code,Ship to Contact First name,Ship to Contact Last name,Ship to Email,Ship to Phone Number,eDelivery Email Address,Tax Status,PO Number,SO Number,Service Price List,Product Category,Product List Price,Service List Factor,Product Family,Smart Account / Virtual Account,Initial Term (Months),Billing Frequency,Automatically Renews For,Subscription ID,Reference Serial Number,Reference Instance Number,LineNum,ParentKey,U_StatusVendor,U_OV
0,UCSC-C220-M4S,"UCS C220 M4 SFF w/o CPU, mem, HD, PCIe, PSU, r...",2024-02-29,FCH2216V13P,5243285545,Major,5243285545,RENEW,NaN,SNT,SNTC 8X5XNBD,CON-SNT-C220M4S,TS,No,203061476,203061476,2021-02-01,2022-01-31,1,NaN,R,Annual,576.13,576.13,32,391.77,NaN,2353270,AMC NETWORKS INC,1011595200,AMC NETWORKS INC,CAMPILLO 2541,PARQUE CHAS,BUENOS AIRES,CIUDAD AUTONOMA DE BUENOS AIRES,AR,1427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-EXEMPT,OV 351 BVS TV SA,106467329,Global PriceList-Latin America,NaN,0,NaN,UCSC,AMC NETWORKS INC-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,1,,,
1,UCSC-RAILB-M4,Ball Bearing Rail Kit for C220 & C240 M4 & M5 ...,NaT,NaN,5243285574,Minor,5243285545,RENEW,NaN,SNT,SNTC 8X5XNBD,NaN,TS,No,203061476,203061476,2021-02-01,2022-01-31,1,NaN,R,NaN,0.00,0.00,0,0.00,NaN,2353270,AMC NETWORKS INC,1011595200,AMC NETWORKS INC,CAMPILLO 2541,PARQUE CHAS,BUENOS AIRES,CIUDAD AUTONOMA DE BUENOS AIRES,AR,1427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-EXEMPT,OV 351 BVS TV SA,106467329,Global PriceList-Latin America,NaN,0,NaN,UCSC,AMC NETWORKS INC-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,2,,,


#### 1.3.4 FORMATEO Y LIMPEZA DE DATOS DE DF BASE

In [27]:
lcr_df = column_filter(lcr_df, lc_filtro_columnas)

In [28]:
lcr_df.head(2)

,ParentKey,LineNum,Product Number,Product Description,Instance Number,PAK/Serial Number,Start Date,End Date,U_StatusVendor,Service Level Description,U_OV,End Customer Name,Target Contract Number,SKU,Last Date of Support
0,,1,UCSC-C220-M4S,"UCS C220 M4 SFF w/o CPU, mem, HD, PCIe, PSU, r...",5243285545,FCH2216V13P,2021-02-01,2022-01-31,,SNTC 8X5XNBD,,AMC NETWORKS INC,203061476,CON-SNT-C220M4S,2024-02-29
1,,2,UCSC-RAILB-M4,Ball Bearing Rail Kit for C220 & C240 M4 & M5 ...,5243285574,NaN,2021-02-01,2022-01-31,,SNTC 8X5XNBD,,AMC NETWORKS INC,203061476,NaN,NaT


> **NOTA:** A partir de ahora `lcr_df` es el "Data Frame base" utilizado para replicarlo en cada contrato

#### 1.3.5 RECORRIDA CABECERA CONTRATO Y ARMADO DE TODAS LAS LINEAS DE CONTRATO
Esta recorrida se utiliza para armar el listado completo de lineas de contrato basándose para ello en la cantidad de contratos cargados en "Cabecera Contrato"

In [29]:
ccr_df

,ContractID,CstmrCode,Status,CntrcType,Renewal,StartDate,EndDate,Descriptio,SrvcType,U_ServType,U_Provider,U_ServiceName,U_OV,U_statusVendor,U_Sla
0,0,CEXT00000005,scs_Approved,ct_SerialNumber,tNO,2021-02-01,2022-01-31,NaN,bst_Regular,NaN,CISCO,SMARTNET,OV 2236 BVS GROUP CORP.,ACTIVE,


In [30]:
# Creacion de un df vacío con las columnas de linea de contrato. Este df luego se utilizará para el armado del CSV.
lcr_total_df = pd.DataFrame(columns=lc_filtro_columnas)

# Recorrida de cabecera de contrato para consolidar las lineas de contrato en lcr_total_df
for i in range(len(ccr_df)):
    ParentKey = ccr_df.loc[i,"ContractID"]
    U_StatusVendor = ccr_df.loc[i,"U_statusVendor"]
    U_OV = ccr_df.loc[i,"U_OV"]
    U_VendorContract = ccr_df.loc[i,"U_statusVendor"]
    U_Sla = ccr_df.loc[i,"U_Sla"]
    U_Provider = ccr_df.loc[i,"U_Provider"]

    lcr_temp_df = lcr_df.copy()
    lcr_temp_df["ParentKey"] = ParentKey
    lcr_temp_df["U_StatusVendor"] = U_StatusVendor
    lcr_temp_df["U_OV"] = U_OV
    lcr_temp_df["U_VendorContract"] = U_VendorContract
    # Identificación x tipo de contrato
    #  Contrato BCARE
    if U_Provider == "BVS":
        lcr_temp_df["Service Level Description"] = U_Sla
        lcr_temp_df["Target Contract Number"] = ""
    
    lcr_total_df = pd.concat([lcr_total_df, lcr_temp_df])

In [35]:
lcr_total_df.sample(5)

,ParentKey,LineNum,ItemCode,ItemName,ManufacturerSerialNum,InternalSerialNum,StartDate,EndDate,U_StatusVendor,U_Sla,U_OV,U_EndCustomer,U_VendorContract,U_ServSKU,U_EoS
,ParentKey,LineNum,itemCode,itemName,ManufSN,InternalSN,StartDate,EndDate,U_StatusVendor,U_Sla,U_OV,U_EndCustomer,U_VendorContract,U_ServSKU,U_EoS
0,0,1,UCSC-C220-M4S,"UCS C220 M4 SFF w/o CPU, mem, HD, PCIe, PSU, r...",5243285545,FCH2216V13P,2021-02-01,2022-01-31,ACTIVE,SNTC 8X5XNBD,OV 2236 BVS GROUP CORP.,AMC NETWORKS INC,203061476,CON-SNT-C220M4S,2024-02-29 00:00:00
29,0,30,UCS-M4-V4-LBL,Cisco M4 - v4 CPU asset tab ID label (Auto-Exp...,5243285557,,2021-02-01,2022-01-31,ACTIVE,SNTC 8X5XNBD,OV 2236 BVS GROUP CORP.,AMC NETWORKS INC,203061476,,2024-02-29 00:00:00
6,0,7,C1UCS-OPT-OUT,Cisco ONE Data Center Compute Opt Out Option,5243285580,,2021-02-01,2022-01-31,ACTIVE,SNTC 8X5XNBD,OV 2236 BVS GROUP CORP.,AMC NETWORKS INC,203061476,,
32,0,33,UCS-HD300G10K12G,300GB 12G SAS 10K RPM SFF HDD,5243285565,W0K2B2BN,2021-02-01,2022-01-31,ACTIVE,SNTC 8X5XNBD,OV 2236 BVS GROUP CORP.,AMC NETWORKS INC,203061476,,
31,0,32,UCSC-HS-C220M4,Heat sink for UCS C220 M4 rack servers,5243285553,,2021-02-01,2022-01-31,ACTIVE,SNTC 8X5XNBD,OV 2236 BVS GROUP CORP.,AMC NETWORKS INC,203061476,,2024-02-29 00:00:00


In [32]:
# lcr_total_df.loc[lcr_total_df["Instance Number"] == 5140307406]

#### 1.3.6 FORMATEO Y LIMPEZA DE DATOS DF TOTAL

In [33]:

lcr_total_df = add_column_format(lcr_total_df, lc_filtro_columnas, lc_cabecera, lc_columnas_definitivas)


MSG: Data-Frame listo para procesar en formato CSV


#### 1.3.7 CREACION DE CSV

In [34]:
# Salida en formato csv
lcr_total_df.to_csv("./cmdb.templates.ok/LINEAS_CONTRATO.csv", index = False)
#lcr_total_df.to_excel("./cmdb.templates.ok/Test_LINEAS_CONTRATO.xls", index = False)
